# Double cart-pole

In [10]:
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    LinearQuadraticRegulator,
    Parser,
    PlanarSceneGraphVisualizer,
    Simulator,
    Linearize,
    StartMeshcat,
    SceneGraph,
    MeshcatVisualizerCpp,
    RigidTransform,
    RotationMatrix,
)
import os
import numpy as np
from underactuated.meshcat_cpp_utils import MeshcatSliders


In [11]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

Meshcat is now available at http://localhost:7003


In [13]:
def simulate_double_cartpole():

    # start construction site of our block diagram
    builder = DiagramBuilder()

    # instantiate the cart-pole and the scene graph
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    Parser(plant).AddModelFromFile("double_cartpole.urdf")
    plant.Finalize()

    # set initial unstable equilibrium point
    context = plant.CreateDefaultContext()
    x_star = [0, 1.5*np.pi, np.pi, 0, 0, 0]
    context.get_mutable_continuous_state_vector().SetFromVector(x_star)

    # Setup slider input
    meshcat.AddSlider('u', min=-30., max=30, step=.1, value=0.0)
    force_system = builder.AddSystem(MeshcatSliders(meshcat,['u']))
    builder.Connect(force_system.get_output_port(),
                    plant.get_actuation_input_port())


    # Setup visualization
    MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
    meshcat.Delete()
    meshcat.Set2dRenderMode(
        X_WC=RigidTransform(RotationMatrix.MakeZRotation(np.pi), [0, 1, 0])
    )

    # finish building the block diagram
    diagram = builder.Build()

    # instantiate a simulator
    simulator = Simulator(diagram)
    simulator.set_publish_every_time_step(False)  # makes sim faster
    
    context = simulator.get_mutable_context()
    context.SetTime(0)
    # context.setContinuousState(np.array([-2, 0.96*np.pi, 0.93*np.pi, 0, 0, 0]))

    # run simulation
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.0)
    meshcat.AddButton('Stop Simulation')
    while meshcat.GetButtonClicks('Stop Simulation') < 1:
        simulator.AdvanceTo(simulator.get_context().get_time() + 1.0)
    meshcat.DeleteAddedControls()

simulate_double_cartpole()
